# NON-LINEAR CLASSIFICATION

#####  All algorithms were designed by Hyungjoo Kim and Dataset was provided at UCL

In [5]:
import numpy as np
import matplotlib.pyplot as plt

from numpy.random import rand

In [3]:
train_label = np.load('./IRIS/iris_train_labels.npy') # [2 1 1 0 2 2 2 0 1...] (96,)
train_samp = np.load('./IRIS/iris_train_samples.npy') # (96,4) sepal length, sepal width, petal length, petal width.
val_label = np.load('./IRIS/iris_val_labels.npy')
val_samp = np.load('./IRIS/iris_val_samples.npy')
# Class
# Iris Setosa for label 0
# Iris Versicolour label 1
# Iris Virginica for label 2
MNIST_train_label = np.load('./MNIST/mnist_train_labels.npy')
MNIST_train_samp = np.load('./MNIST/mnist_train_samples.npy')  # (44800, 28*28)
MNIST_val_label = np.load('./MNIST/mnist_val_labels.npy')
MNIST_val_samp = np.load('./MNIST/mnist_val_samples.npy')

**Task 1: Implement classification based on logistic regression using GD by implementing the gradient function deLogistic(preds, X, Y) and optimizing using GD. preds are the prediction from the model, X are the data and Y are the labels. Propose a function interface for your implementation of the gradient descent algorithm.**

In [24]:
def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

def loss_function(preds, Y):
    return np.sum((preds - Y)**2) / len(Y)

def de_Logistic(preds, X, Y):
    dw = (1 / len(Y)) * np.matmul(X.T, preds - Y)
    db = (1 / len(Y)) * np.sum(preds - Y)
    return dw, db

def Logistic_train_classifier(X, Y, learning_rate, max_iter):
    Y = np.expand_dims(Y, axis = 1)
    w = np.random.rand(X.shape[1], 1)
    b = 0
    
    history = []
    best_loss = float('+inf')
    best_w = None
    best_b = None
    checkpoint_step = max_iter // 10 if max_iter >= 5 else 1
    
    for i in range(max_iter):
        z = np.matmul(X, w)
        prediction = sigmoid(z)
        loss = loss_function(prediction, Y)
        if loss < best_loss:
            best_loss = loss
            best_w = w
            best_b = b
        
        dw, db = de_Logistic(prediction, X, Y)
        w = w - learning_rate * dw
        b = b - learning_rate * db
        history.append(loss)
        
        if (i + 1) % checkpoint_step == 0:
            binary_pred = (prediction >= 0.5) * 1  # 0 or 1
            correct = 0
            for k in range(len(binary_pred)):
                if binary_pred[k] == Y[k]:
                    correct += 1
            accuracy = correct / len(binary_pred)
            print('{} / {} Current loss is {}, Traning accuracy is {}'.format(i + 1, max_iter, loss, accuracy))
    return history, best_w, best_b, accuracy

# Setosa = 0, Vesicolour and Virginica = 1
binary_train_label = np.array([1 if x != 0 else 0 for x in train_label])
_, best_w, best_b, _ = Logistic_train_classifier(train_samp, binary_train_label, 0.001, 1000)

100 / 1000 Current loss is 0.29210562276506374, Traning accuracy is 0.6666666666666666
200 / 1000 Current loss is 0.2279424166160572, Traning accuracy is 0.6666666666666666
300 / 1000 Current loss is 0.1527039094729769, Traning accuracy is 0.6666666666666666
400 / 1000 Current loss is 0.10864737225952444, Traning accuracy is 0.6770833333333334
500 / 1000 Current loss is 0.08956502524958765, Traning accuracy is 0.9479166666666666
600 / 1000 Current loss is 0.07984472434881174, Traning accuracy is 0.9791666666666666
700 / 1000 Current loss is 0.07328503200342673, Traning accuracy is 0.9895833333333334
800 / 1000 Current loss is 0.0680390364333412, Traning accuracy is 1.0
900 / 1000 Current loss is 0.06351621127950516, Traning accuracy is 1.0
1000 / 1000 Current loss is 0.05948988217387365, Traning accuracy is 1.0


**Task 2: Implement classification based on hinge loss using GD by implementing the gradient function deHinge(preds, W, X, Y) and optimizing using GD. preds are the prediction from the model, W describes the model parameters, X is the data and Y represents the labels. Propose a function interface for your implementation of the gradient descent algorithm.**

In [32]:
def hinge_loss_function(s, Y):
    n_samples = len(s)
    correct_score = s[list(range(n_samples)), Y]
    correct_score = np.expand_dims(correct_score, axis = 1)  # Save correct score from the labels
    
    s = s - correct_score + 1  # sj - syi + 1 (syi = correct score)
    s[list(range(n_samples)), Y] = 0  # Correct scores to make zero because not into the hinge loss
    loss = np.sum(np.fmax(s, 0)) / n_samples
    return s, loss

def deHinge(preds, w, X, Y):
    s = preds
    dw = np.zeros(w.shape)
    X_mask_for_margin = np.zeros(s.shape)  # n * class
    X_mask_for_margin[s > 0] = 1  # positive? -> 1
    X_mask_for_margin[np.arange(len(s)), Y] = -np.sum(X_mask_for_margin, axis = 1)  # Counts only positive
    dw = X.T.dot(X_mask_for_margin)
    dw = dw / len(s)
    return dw

def Hinge_train_classifier(X, Y, learning_rate, max_iter):
    w = np.random.rand(X.shape[1], len(set(Y)))
    b = 0
    
    history = []
    best_loss = float('+inf')
    best_w = None
    checkpoint_step = max_iter // 10 if max_iter >= 5 else 1
    for i in range(max_iter):
        score = np.matmul(X, w)  # X(N, dims) w(dims, 3)
        s, loss = hinge_loss_function(score, Y)
        if loss < best_loss:
            best_w = w
            best_loss = loss
            
        dw = deHinge(s, w, X, Y)
        w = w - learning_rate * dw
        history.append(loss)
        if (i + 1) % checkpoint_step == 0:
            prediction = np.argmax(score, axis = 1)  # Multi-class
            correct = 0
            for k in range(len(prediction)):
                if prediction[k] == Y[k]:
                    correct += 1
            accuracy = correct / len(prediction)
            print("{} / {} Current loss is {}, Training accuracy is {}".format(i + 1, max_iter, loss, accuracy))
    return history, best_w

_, _ = Hinge_train_classifier(train_samp, train_label, 0.1, 3000)

300 / 3000 Current loss is 0.0594060582797743, Training accuracy is 0.9895833333333334
600 / 3000 Current loss is 0.04838461716198697, Training accuracy is 0.9895833333333334
900 / 3000 Current loss is 0.04049637931476736, Training accuracy is 0.9895833333333334
1200 / 3000 Current loss is 0.03493853617260379, Training accuracy is 1.0
1500 / 3000 Current loss is 0.03051460640992427, Training accuracy is 1.0
1800 / 3000 Current loss is 0.028495786986287668, Training accuracy is 1.0
2100 / 3000 Current loss is 0.02791117558777122, Training accuracy is 1.0
2400 / 3000 Current loss is 0.02448372618604, Training accuracy is 1.0
2700 / 3000 Current loss is 0.022513018312925164, Training accuracy is 1.0
3000 / 3000 Current loss is 0.02113107386848015, Training accuracy is 1.0


**Task 3: Implement kernel SVM function ksvm(kernel, X, Y, xtest). The function takes as input a kernel function kernel, training data X, Y and a set of test points xtest. The function returns the set of support vectors along with the predicted labels. You are allowed to use scipy optimization library to solve the quadratic problem of SVM.**

In [ ]:
def ksvm(kernel, X, Y, xtest):
    support_idx = ksvm.support_
    support_vectors = X_train[support_idx]
    intercept = ksvm.intercept_
    surpport_lb = Y_train[support_idx] * np.abs(ksvm.dual_coef_)
    
    return support_vectors, support_lb

def kernel(xi, xj):
    poly_kernel_function = ((np.dot(xi.T, xj) + 1)**d)
    return poly_kernel_func
    
def kernel_rbf(xi, xj):
    d1 = xi - xj  # vector
    d2 = np.dot(d1, d1)   # vector squared = number
    
    return np.exp(-d2 / 2)

def predict(xj, support_vect, support_lb):
    kv = np.apply_along_axis(lambda xi: kernel_rbf(xi, xj), 1, support_vectors)
    preds = kv * support_lb
    
    return np.sum(preds)

train_samp /= 255.0
train_label /= 255.0

support_vecs, support_lb = ksvm(kernel, train_samp, val_samp, train_label)
kernel = kernel(support_vecs, support_lb)

preds = np.apply_along_axis(lambda x_j : predict(x_j, support_vect, support_lb), 1, X_test[:50] / 255.0)
preds = 2 * (preds > 0) - 1
plot_confusion_matrix(Y_test[:50], preds)